In [36]:
from tda import auth, client
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import psycopg2
import os
import json

load_dotenv()

True

In [32]:
TD_KEY = os.getenv('CONSUMER_KEY')
ACC_NUMBER = os.getenv('ACC_NUMBER')
REDIRECT_URI = os.getenv('REDIRECT_URI')
FOLDER_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
TOKEN_PATH = os.path.join(FOLDER_PATH, 'tokens/token.pickle')
API_KEY = TD_KEY + '@AMER.OAUTHAP'

def connect_client(api_key, redirect_uri, token_path, webdriver_func):
    client = auth.easy_client(
        api_key = api_key,
        redirect_uri = REDIRECT_URI,
        token_path = TOKEN_PATH,
        webdriver_func = webdriver_func,
    )
    return client

def get_webdriver(path = None):
    if not path:
        path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
        path = os.path.join(path, 'chromedriver')
    return webdriver.Chrome(path)

In [34]:
# Load connection to db
conn = psycopg2.connect('dbname=securities user=postgres')
cur = conn.cursor()

# Extract list of all security names
cur.execute('SELECT symbol FROM holdings')
all_securities = [x[0] for x in cur.fetchall()]

# Start TDA Client
CLIENT = connect_client(API_KEY, REDIRECT_URI, TOKEN_PATH, get_webdriver)

In [56]:
# Get all securities in population list
response = CLIENT.get_quotes(','.join(all_securities)).json()
response_keys = response.keys()

In [57]:
len(response)

197

In [58]:
# Build list of values that I want to insert into my database

df_list = list()

for key in response_keys:
    temp_list = [
        key,
        response[key]['description'],
        response[key]['assetType'],
        response[key]['assetMainType'],
    ]
    df_list.append(temp_list)
    
df_securities = pd.DataFrame(df_list, columns = ['symbol', 'description', 'asset_type', 'main_type'])

In [59]:
df_securities

,symbol,description,asset_type,main_type
0,ISHG,iShares 1-3 Year International Treasury Bond ETF,ETF,EQUITY
1,TLT,iShares 20+ Year Treasury Bond ETF,ETF,EQUITY
2,IEI,iShares 3-7 Year Treasury Bond ETF,ETF,EQUITY
3,IEF,iShares 7-10 Year Treasury Bond ETF,ETF,EQUITY
4,USHY,iShares Broad USD High Yield Corporate Bond ETF,ETF,EQUITY
...,...,...,...,...
192,IVOG,Vanguard S&P Mid-Cap 400 Growth ETF,ETF,EQUITY
193,BSV,Vanguard Short-Term Bond ETF,ETF,EQUITY
194,VGSH,Vanguard Short-Term Treasury ETF,ETF,EQUITY
195,VTIP,Vanguard Short-Term Inflation-Protected Securi...,ETF,EQUITY


In [61]:
df_securities.to_csv('./pop_securities.csv')

In [63]:
# Alternative - query straight from existing DB
cur.execute('SELECT * FROM holdings;')
query_result = cur.fetchall()

In [64]:
query_result[0]

('ISHG',
 'iShares 1-3 Year International Treasury Bond ETF',
 'ETF',
 'EQUITY',
 0,
 0.0)

In [66]:
df_population = pd.DataFrame(query_result, columns = ['symbol', 'description', 'asset_type', 'main_type', 'quantity', 'target'])

In [67]:
df_population.head()

,symbol,description,asset_type,main_type,quantity,target
0,ISHG,iShares 1-3 Year International Treasury Bond ETF,ETF,EQUITY,0,0.0
1,TLT,iShares 20+ Year Treasury Bond ETF,ETF,EQUITY,0,0.0
2,IEI,iShares 3-7 Year Treasury Bond ETF,ETF,EQUITY,0,0.0
3,IEF,iShares 7-10 Year Treasury Bond ETF,ETF,EQUITY,0,0.0
4,USHY,iShares Broad USD High Yield Corporate Bond ETF,ETF,EQUITY,0,0.0


In [70]:
df_population.to_csv('./pop_securities.csv', index = False)